In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm
import glob
import pickle
import random

from keras import applications
from keras.preprocessing import image
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
# from keras.applications.inception_v3 import InceptionV3, preprocess_input

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split


from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
# from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pickle

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
# inception_resnet_v2.InceptionResNetV2

from keras.applications.xception import preprocess_input as preprocess_xception

Using TensorFlow backend.


In [2]:
animals_list = []
for each in glob.glob('data/*'):
    if each.split('\\')[1] != 'human':
        animals_list.append(each.split('\\')[1])

In [3]:
animals_list

['buffalo',
 'chihuahua',
 'chimpanzee',
 'dugong',
 'elephant',
 'giant_panda',
 'grizzly+bear',
 'hippopotamus',
 'horse',
 'lion',
 'malayan_tiger',
 'orang_utan']

In [4]:
y_train = []

In [5]:
file_paths = []
for i in range(len(animals_list)):
    files_list = glob.glob('data/'+animals_list[i]+'/*')
    for each in files_list:
        file_paths.append('data/'+animals_list[i] + '/'+each.split('\\')[-1])
        y_train.append(animals_list[i])

In [6]:
human_list = []
for each in glob.glob('data/human'+'/*'):
    human_list.append('data/human/'+each.split('\\')[-1])

In [7]:
required_human = random.sample(human_list, 400)
required_human_target = ['human' for each in range(400)]

In [8]:
len(y_train), len(file_paths)

(7739, 7739)

In [9]:
y_train = y_train + required_human_target
file_paths = file_paths + required_human

In [10]:
len(y_train), len(file_paths)

(8139, 8139)

In [11]:
##define VARIABLES

HEIGHT=400
WIDTH=400
VAL_SPLIT = 0.2 #Percentage of training examples used for validation

In [12]:
def read_img(img_path,size):
    img = image.load_img(img_path, target_size=size)
    img = image.img_to_array(img)
    return img

In [13]:
# # load data. Use pickle file if they exist
x_train = np.zeros((len(y_train), HEIGHT, WIDTH, 3), dtype='float32')

train_img, test_img = [],[]
i = 0
for img_path in tqdm(file_paths):
    x = read_img(img_path,(HEIGHT, WIDTH))
    x_train[i] = preprocess_xception(np.expand_dims(x.copy(), axis=0))
    i = i+1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8139/8139 [04:17<00:00, 31.63it/s]


In [14]:
targets = pd.Series(y_train)
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [15]:
one_hot.columns.tolist()

['buffalo',
 'chihuahua',
 'chimpanzee',
 'dugong',
 'elephant',
 'giant_panda',
 'grizzly+bear',
 'hippopotamus',
 'horse',
 'human',
 'lion',
 'malayan_tiger',
 'orang_utan']

In [19]:
split = 0.8
split_num = int(x_train.shape[0]*0.8)

x_tr = x_train[:split_num]
y_tr = one_hot_labels[:split_num]

x_val = x_train[split_num:]
y_val = one_hot_labels[split_num:]

print(x_tr.shape, y_tr.shape)
print(x_val.shape, y_val.shape)

(7343, 400, 400, 3) (7343, 10)
(1836, 400, 400, 3) (1836, 10)


In [16]:
X_tr, X_te, y_tr, y_te = train_test_split(x_train, one_hot_labels, test_size=0.2, random_state=42)

In [17]:
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
train_x_bf = xception_bottleneck.predict(X_tr, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(X_te, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

1628/1628 [==============================] - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 26s 16ms/step
Xception train bottleneck features shape: (6511, 2048) size: 13,334,528
Xception valid bottleneck features shape: (1628, 2048) size: 3,334,144


In [18]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=129)
logreg.fit(train_x_bf, (y_tr * range(13)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_x_bf)
valid_preds = logreg.predict(valid_x_bf)
print('Validation Xception LogLoss {}'.format(log_loss(y_te, valid_probs)))
print('Validation Xception Accuracy {}'.format(accuracy_score((y_te * range(13)).sum(axis=1), valid_preds)))

Validation Xception LogLoss 0.025019983769086304
Validation Xception Accuracy 0.995085995085995


In [19]:
model_json = xception_bottleneck.to_json()
with open("model_13_animal_danger_human.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
xception_bottleneck.save_weights("model_13_animal_danger_human.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
pickle.dump(logreg, open('logreg_13_animal_danger_human.model', 'wb'))